In [14]:
# 필요한 라이브러리 설치
!pip install torch pandas joblib

In [15]:
# 모델과 벡터화기 파일 ID 설정
FILE_ID_MODEL = "1WDD0Ptkqn5x7lXXlvPmp4IQSkIs3HIEW"
FILE_ID_VECT  = "14s9YqXzfvoHSCDcAdr02MZ1pceL9LIRG"

In [16]:
# gdown으로 다운로드
!gdown --id $FILE_ID_MODEL -O news_model.pth
!gdown --id $FILE_ID_VECT  -O tfidf.pkl

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1WDD0Ptkqn5x7lXXlvPmp4IQSkIs3HIEW
To: /content/news_model.pth
100% 10.2M/10.2M [00:00<00:00, 28.2MB/s]
/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=14s9YqXzfvoHSCDcAdr02MZ1pceL9LIRG
To: /content/tfidf.pkl
100% 428k/428k [00:00<00:00, 26.1MB/s]


In [17]:
import joblib
import torch
import torch.nn as nn

# 벡터화기 로드
vectorizer = joblib.load("tfidf.pkl")

class NewsClassifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, 256)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(256, 2)

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        return self.fc2(x)

# 모델 생성 및 가중치 로드
model = NewsClassifier(input_dim=vectorizer.max_features)
model.load_state_dict(torch.load("news_model.pth"))
model.eval()

NewsClassifier(
  (fc1): Linear(in_features=10000, out_features=256, bias=True)
  (relu): ReLU()
  (dropout): Dropout(p=0.3, inplace=False)
  (fc2): Linear(in_features=256, out_features=2, bias=True)
)

In [18]:
def predict_news(text_list):
    """
    text_list: 문자열 뉴스 문장 리스트
    return: 관심 있음/없음 예측 결과
    """
    X_vec = vectorizer.transform(text_list).toarray()
    X_tensor = torch.tensor(X_vec, dtype=torch.float32)

    with torch.no_grad():
        outputs = model(X_tensor)
        preds = outputs.argmax(1).numpy()

    return preds

In [19]:
# title + description 합친 실제 뉴스 예시
sample_news = [
    "AI 챗봇 출시 " + "최신 AI 챗봇이 출시되어 사용자와 자연스러운 대화를 지원한다.",
    "삼성전자 주가 급등 " + "오늘 주식 시장에서 삼성전자의 주가가 크게 상승했다.",
    "국제 협약 체결 " + "국제 정치 회담에서 새로운 협약이 공식 체결되었다.",
    "AI 기술 논문 발표 " + "한 대학 연구팀이 최신 AI 알고리즘 논문을 발표했다.",
    "경제 성장률 발표 " + "한국의 올해 3분기 경제 성장률이 공개되었다.",
    "IT 신제품 공개 " + "삼성전자와 애플이 새로운 스마트폰 모델을 공개했다.",
    "사회적 거리두기 완화 " + "정부가 사회적 거리두기 조치를 단계적으로 완화한다고 발표했다.",
    "국제 스포츠 대회 개최 " + "2025년 국제 스포츠 대회가 성공적으로 개최되었다.",
    "AI 스타트업 투자 유치 " + "국내 AI 스타트업이 50억 원 규모 투자 유치에 성공했다.",
    "정치 토론회 개최 " + "정당 간 정책 토론회가 오늘 국회에서 열렸다."
]

preds = predict_news(sample_news)

for news, pred in zip(sample_news, preds):
    label = "관심 있음" if pred == 1 else "관심 없음"
    print(f"[{label}] {news}")


[관심 있음] AI 챗봇 출시 최신 AI 챗봇이 출시되어 사용자와 자연스러운 대화를 지원한다.
[관심 없음] 삼성전자 주가 급등 오늘 주식 시장에서 삼성전자의 주가가 크게 상승했다.
[관심 없음] 국제 협약 체결 국제 정치 회담에서 새로운 협약이 공식 체결되었다.
[관심 있음] AI 기술 논문 발표 한 대학 연구팀이 최신 AI 알고리즘 논문을 발표했다.
[관심 있음] 경제 성장률 발표 한국의 올해 3분기 경제 성장률이 공개되었다.
[관심 없음] IT 신제품 공개 삼성전자와 애플이 새로운 스마트폰 모델을 공개했다.
[관심 있음] 사회적 거리두기 완화 정부가 사회적 거리두기 조치를 단계적으로 완화한다고 발표했다.
[관심 없음] 국제 스포츠 대회 개최 2025년 국제 스포츠 대회가 성공적으로 개최되었다.
[관심 있음] AI 스타트업 투자 유치 국내 AI 스타트업이 50억 원 규모 투자 유치에 성공했다.
[관심 없음] 정치 토론회 개최 정당 간 정책 토론회가 오늘 국회에서 열렸다.
